# Laser pulse propagation in optical fiber

This notebook shows a simple, scalar implementation of the [finite difference](https://en.wikipedia.org/wiki/Finite_difference_method) for solving the [Nonlinear Schrödinger Equation](https://en.wikipedia.org/wiki/Nonlinear_Schrödinger_equation).

$\frac{\partial A}{\partial z}=-\frac{\alpha}{2}A+i \frac{\beta_2}{2} \frac{\partial^2 A}{\partial t^2}-i \gamma(|A|^2A)$

This nonlinear partial differential equation models how the envelope and phase of light pulse changes when propagating through a single mode optical fiber, when taking power attenuation ($\alpha$), group velocity dispersion ($\beta_2$)and waveguide nonlinearity ($\gamma$) causing self-phase modulation (SPM) into account.

## Importing the modules

In [ ]:
from libraries import *
from variables import *
from functions import *
from classes import *

## Initialize class about simulation parameters

In [ ]:
sim_config = SIM_config(simunit,N,dt,wavelength0)

## Initialize class about plot parameters

In [ ]:
plot_config = PLOT_config(plotunit,N,dt,wavelength0,duration)

## Testing the defined functions

In [ ]:
# Initialize Gaussian pulse in the time domain
testPulse = GaussianPulseTime(plot_config.t,sim_config.amplitude,plot_config.duration) / sim_config.amplitude

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.title("Initial pulse")
ax1.plot(plot_config.t,getPower(testPulse)/np.max(getPower(testPulse)),'g-')
ax2.plot(plot_config.t,abs(testPulse)/np.max(abs(testPulse)),'b-')
ax1.set_xlabel("Time " + timedictionary[plotunit])
ax1.set_ylabel("Power [arbitrary unit]", color='g')
ax2.set_ylabel("Amplitude [arbitrary unit]", color='b')
ax1.set_ylim(0,1)
ax2.set_ylim(0,1)
plt.xlim(-5*plot_config.duration,5*plot_config.duration)
savePlot('initial pulse')
plt.show()

## Testing the defined spectrum functions

In [ ]:
# Initialize Gaussian pulse in the frequency domain
testSpectrumTheory = GaussianPulseFrequency(plot_config.f,plot_config.frequency0,sim_config.amplitude,plot_config.duration) / sim_config.amplitude
# Getting spectrum from pulse
testSpectrum=getSpectrumFromPulse(plot_config.t,plot_config.f,testPulse)
# Getting Gaussian spectrum
testSpectrum_2=GaussianSpectrum(plot_config.t,plot_config.f,sim_config.amplitude,1/(plot_config.duration)) / sim_config.amplitude
plt.figure()
plt.title("Spectrum of initial pulse")
plt.plot(plot_config.f,getPower(testSpectrumTheory)/np.max(getPower(testSpectrumTheory)),label="Spectrum of the theory")
plt.plot(plot_config.f_rel,getPower(testSpectrum)/np.max(getPower(testSpectrum)),label="Spectrum of testPulse")
plt.plot(plot_config.f_rel,getPower(testSpectrum_2)/np.max(getPower(testSpectrum_2)),label="Spectrum generated with GaussianSpectrum")
plt.axis([-1/plot_config.duration + plot_config.frequency0,1/plot_config.duration + plot_config.frequency0,0,1])
plt.xlabel("Frequency " + frequencydictionary[plotunit])
plt.ylabel("Power spectral density [arbitrary unit]")
plt.legend()
savePlot('spectrum of the initial pulse')
plt.show()

## Testing the FWHM function

In [ ]:
FWHM_frequency=FWHM(plot_config.f,getPower(testSpectrum))
FWHM_time=FWHM(plot_config.t,getPower(testPulse))
product=FWHM_frequency*FWHM_time
print(f"The product of the pulse and the spectrum's FWHM is {product}")

## Initialize class about fiber

In [ ]:
fiber=Fiber_config(fiberunit,nsteps,Length,gammaconstant,beta2,alpha_dB_per_m)

# Run Simulation

In [ ]:
pulseMatrix, spectrumMatrix = Simulation(fiber,sim_config,testPulse,'EFORK3')
testPulse0=np.copy(pulseMatrix[0,:])
testSpectrum0=np.copy(spectrumMatrix[0,:])
testPulseFinal=np.copy(pulseMatrix[-1,:])
testSpectrumFinal=np.copy(spectrumMatrix[-1,:])

## Initialize functions for plotting the results

In [ ]:
#%%script false
FWHM_frequency_final=FWHM(plot_config.f_rel,getPower(testSpectrumFinal))
#plotFirstAndLastPulse(pulseMatrix,plot_config)
#plotPulseMatrix2D(pulseMatrix,fiber,plot_config)
#plotFirstAndLastSpectrum(spectrumMatrix,plot_config,FWHM_frequency_final)
#plotPSDWavelength(spectrumMatrix,plot_config)
#plotSpectrumMatrix2D(spectrumMatrix,fiber,sim_config,FWHM_frequency_final)
#plotSpectrogram(plot_config, pulseMatrix[0,:], 2000, 2000, sim_config.time_step, label='initial')
#plotSpectrogram(plot_config, pulseMatrix[-1,:], 2000, 2000, sim_config.time_step, label='final')